# RNN으로 금융앱 리뷰 분석하기

In [1]:
# mecab설치
!pip install konlpy

In [5]:
from konlpy.tag import Mecab # 우분투에 mecab설치한적있어서 경로맞으면 바로 실행됨
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical

In [7]:
train_data= pd.read_csv('./data/bank_app_reviews_train.csv')
test_data= pd.read_csv('./data/bank_app_reviews_test.csv')

In [ ]:
# 데이터 제대로 들어왔는지 df 찍어보고 다음으로.

### 1) 특수문자 제거 함수 정의

In [8]:
import re
def clean_text(text):
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s]','',text) # 한글, 영문, 숫자
    cleaned = re.sub(r'\s+',' ',cleaned) # 연속된 공백을 하나의 공백으로
    return cleaned.strip()

### 사용자리뷰 내용의 특수문자 제거

In [10]:
train_data['사용자리뷰'] = train_data['사용자리뷰'].apply(clean_text)
train_data['사용자리뷰']

0                  엄빠 폰에 설치해드렸는데 두분 다 쓰기 편하다고 하시네요 저도 쓰고있음
1        Cd기 축소 연장하려면 이 어플 깔라는데 왜 30퍼에서 안깔리는지 아니 애초에 슈퍼...
2                             서비스가 통합되어 있어서 점점 사용빈도가 높아지네요
3                                                카드 충천이 안됌
4                                              알뜰폰 인증이 안돼요
                               ...                        
22241                                      어플 시작오류가 가끔 나네요
22242                              업데이트 후에 계좌이체하는 게 자주 튕겨요
22243                                                  굳굳굳
22244                                  오래시간 같이한 나의은행 든든합니다
22245    라이브 광고 전보다 더 안좋네 아무리 주기 싫어도 겨우 3원줄거면서 너무 쪼잔하게 ...
Name: 사용자리뷰, Length: 22246, dtype: object

### is_good 컬럼 추가, 평점4점 이상은 긍정(1) 3이하는 부정(0)

In [9]:
train_data['is_good']=train_data['평점'].apply(lambda x: 1 if x>=4 else 0)
train_data['is_good']

0        1
1        0
2        1
3        0
4        0
        ..
22241    0
22242    0
22243    1
22244    1
22245    0
Name: is_good, Length: 22246, dtype: int64

### 토큰화

In [12]:
mecab=Mecab()
mecab.morphs(train_data['사용자리뷰'][0]) # 형태소쪼개기

['엄',
 '빠',
 '폰',
 '에',
 '설치',
 '해',
 '드렸',
 '는데',
 '두',
 '분',
 '다',
 '쓰',
 '기',
 '편하',
 '다고',
 '하',
 '시',
 '네요',
 '저',
 '도',
 '쓰',
 '고',
 '있',
 '음']

In [ ]:
# 혹시 안된다면
# ! pip install mecab-python3 진행

### 전체문장 토큰화 후 tokenized_docs에 저장

# LSTM과 CNN 조합 모델로 분석

In [13]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, GlobalMaxPooling1D

In [ ]:
# 모델 구성
lstm_cnn = Sequential()
lstm_cnn.add(Input(shape=(max_length,)))
lstm_cnn.add(Embedding(input_dim=word_size, output_dim=128))
lstm_cnn